In [0]:
from utils.routing_helpers import greedy_assign_priority, compute_kpis

CATALOG = "zg"
SCHEMA = "production_scheduling_demo"
MODEL_NAME = f"{CATALOG}.{SCHEMA}.predictive_routing_model"
PLANNING_DAYS = 28  # scheduling horizon (can adjust)
SEED = 42

In [0]:
# load baseline & candidates
cand_pd = spark.table(f"{CATALOG}.{SCHEMA}.candidate_routes_scored").toPandas()
machines_pd = spark.table(f"{CATALOG}.{SCHEMA}.machines_catalog").toPandas()
assigned_baseline_pd = spark.table(f"{CATALOG}.{SCHEMA}.assigned_baseline").toPandas()

In [0]:
kpi_baseline = compute_kpis(assigned_baseline_pd, machines_pd, PLANNING_DAYS)

print("BASELINE KPIs")
for k, v in kpi_baseline.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

In [0]:
down_machines = ["ELEC-ASSEM-2"]
PLANNING_DAYS = 28

assigned_scenario, caps_scenario = greedy_assign_priority(cand_pd, machines_pd, PLANNING_DAYS, down_machines)

kpi_scenario = compute_kpis(assigned_scenario, machines_pd, PLANNING_DAYS)
print("Machine Down Scenario KPIs")
for k, v in kpi_scenario.items():
    print(f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")

In [0]:
delta = {
    "profit_delta": kpi_scenario["expected_profit"] - kpi_baseline["expected_profit"],
    "ontime_delta": kpi_scenario["expected_ontime_deliveries"] - kpi_baseline["expected_ontime_deliveries"],
    "utilization_delta": kpi_scenario["factory_capacity_utilization"] - kpi_baseline["factory_capacity_utilization"],
    "orders_delta": kpi_scenario["expected_orders_completed"] - kpi_baseline["expected_orders_completed"]
}

print("SCENARIO KPIs:", kpi_scenario)
print("DELTAS:", delta)